# Actividad: Clasificación con máquina de vectores de soporte y redes de neuronas (Mobile Price Classification)

## Introducción

El problema orginal tiene una breve descripción contextual:

> Bob has started his own mobile company. He wants to give tough fight to big companies like Apple,Samsung etc.
>
> He does not know how to estimate price of mobiles his company creates. In this competitive mobile phone market you cannot simply assume things. To solve this problem he collects sales data of mobile phones of various companies.
> 
> Bob wants to find out some relation between features of a mobile phone(eg:- RAM,Internal Memory etc) and its selling price. But he is not so good at Machine Learning. So he needs your help to solve this problem.
>
> In this problem you do not have to predict actual price but a price range indicating how high the price is.

La variable objetivo es "price_range". Solo [train.csv](https://www.kaggle.com/iabhishekofficial/mobile-price-classification#train.csv).

### Bibliotecas a utilizar

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV

## Carga de dataset y análisis descriptivo de datos

In [ ]:
df_train = pd.read_csv("ds/train.csv")

Según los datos proporcionados no existen valores perdidos. Además todas las columnas tienen valores numéricos.

In [ ]:
df_train.info()

In [ ]:
df_train.describe().transpose() 

Aún así parece que no todas las variables son realmente numéricas, pues no caen en valore continuos. Como ejemplo es posible observar la columna objetivo, la cual por medio de números distingue cuatro categorías. Debido a la falta de metadatos en el dataset no es posible saber con certeza a que corresponde cada una, sin embargo, una clasificación tradicional es: el segmento de entra (0), la gama media (1), la gama alta (2) y la gama premium (3).

In [ ]:
sns.countplot(x='price_range', data=df_train)
plt.title('Frecuencias de price_range', fontsize=12)

### Variables categóricas

Es importante observar que datos son categóricos

In [ ]:
for i in df_train.columns:
    print(f'Valores posibles de {i.title()}: {df_train[i].unique()}')

Las variables categóricas parecen ser: `blue`, `dual_sim`, `four_g`, `three_g`, `touch_screen`, `wifi` y `price_range`. Debido a que `price_range` es nuestra variable objetivo, en seguida se grafican las frecuencias de las otras variables categóricas tomando en cuenta el rango de precios.

In [ ]:
cv=['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi']
fig=plt.figure(figsize=(24,12))
plt.title('Frecuencias de variables categóricas tomando en cuenta el rango de precios',fontdict={'fontsize':20})
plt.axis('off')

for i in range(len(cv)):
    fig.add_subplot(2,3,i+1)
    sns.countplot(data=df_train,x=cv[i], hue='price_range')
    plt.legend(bbox_to_anchor=(1.02, 1), borderaxespad=0)

Es posible observar que los teléfonos más costosos cuentan con más de estas características, salvo en el caso de `touch_screen`, donde no hay diferencia frecuencial visible. La variable `three_g` parace ser más significativa ya que la división es un poco más clara. También hay que matizar que esta diferencia no es suficiente para distinguir con invariablemente las categorías de rango de precio.

### Variables numéricas

Se repetirá este procedimiento para las variables que son efectivamente numéricas.

In [ ]:
nv=['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time']
fig=plt.figure(figsize=(24,30))
plt.title('Frecuencia de variables numéricas tomando en cuenta el rango de precios')
plt.axis('off')

for i in range(len(nv)):
    fig.add_subplot(7,2,i+1)
    sns.kdeplot(data=df_train, x=nv[i], hue='price_range')

La dentro de las gráficas anteriores lo más destacado es aquella que representa la variable `ram`. En dicha representación se distinguen muy claramente los segmentos de precio, es posible pensar que que la variable `ram`, tenga una correlación alta respecto a `price_range`, mientras otras como `clock_speed` sean de muy poca relevancia.

## Matriz de correlación

In [ ]:
plt.figure(figsize=(20,8),dpi=80)
corrmat = df_train.corr()
sns.heatmap(corrmat, cmap='coolwarm', vmax=.8, fmt='.1f', annot=True)

In [ ]:
df_train.corr()['price_range'].sort_values(ascending=False)[1:21]

## SVM Lineal

In [ ]:
train, test = train_test_split(df_train, test_size=0.2)
predictors = cv + nv
#predictors = ['ram', 'battery_power', 'px_width', 'px_height', 'int_memory']
target = ['price_range']

In [ ]:
parameters = [{'kernel': ['rbf', 'linear'], 'gamma': ['scale', 'auto'], 'C': [0.5, 1.0, 1.5, 2.0, 2.5]}]
svmc = GridSearchCV(svm.SVC(decision_function_shape='ovr'), param_grid=parameters, cv=25, scoring='accuracy')
svmc.fit(train[predictors], np.ravel(train[target]))
svmc.best_params_

In [ ]:
svm_predicted = svmc.predict(test[predictors])
crosstab = sns.heatmap(pd.crosstab(test[target].values.ravel(), svm_predicted),
                cmap='gist_yarg', vmax=.8, fmt='g', annot=True)
crosstab.set_xlabel('Predicciones')
crosstab.set_ylabel('Reales')
print("Accuracy: ", metrics.accuracy_score(svm_predicted, np.ravel(test[target])))

In [ ]:
print(metrics.classification_report(test[target].values.ravel(), svm_predicted))

## Red Neuronal